# Improve word2vec model through some optimization

### Download and save word2vec in bin file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

directory_path = '/content/drive/MyDrive/FYP TestData'
directory_files = os.listdir(directory_path)

for folder in directory_files:
  print(folder)

TestData.gsheet
TestData.csv
FYPTesting.csv


In [ ]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
pretrained_w2v = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
pretrained_w2v.save("/content/drive/MyDrive/Gensim Embeddings/word2vec.model")

### Load in spcay to test sentence similarity

In [ ]:
print(pretrained_w2v.index_to_key[:10], pretrained_w2v.vectors[:10])

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said'] [[ 1.12915039e-03 -8.96453857e-04  3.18527222e-04 ... -1.56402588e-03
  -1.23023987e-04 -8.63075256e-05]
 [ 7.03125000e-02  8.69140625e-02  8.78906250e-02 ... -4.76074219e-02
   1.44653320e-02 -6.25000000e-02]
 [-1.17797852e-02 -4.73632812e-02  4.46777344e-02 ...  7.12890625e-02
  -3.49121094e-02  2.41699219e-02]
 ...
 [-1.72851562e-01  2.79296875e-01  1.06933594e-01 ...  1.23046875e-01
   1.29882812e-01 -1.82617188e-01]
 [-2.49023438e-02  2.19726562e-02 -3.54003906e-02 ...  1.08032227e-02
  -1.87988281e-02 -6.88476562e-02]
 [-9.09423828e-03 -4.41894531e-02  9.96093750e-02 ...  1.44531250e-01
   1.80664062e-01 -8.69140625e-02]]


In [ ]:
print(len(pretrained_w2v.index_to_key))

3000000


In [ ]:
# !unzip pretrained_w2v_model.zip

In [ ]:
# from gensim.models import KeyedVectors
# pretrained_w2v= KeyedVectors.load("pretrained_w2v_model.bin")

In [ ]:
!pip install spacy

In [ ]:
del nlp
del vocab

In [ ]:
import spacy

nlp = spacy.blank("en")

# Create a spaCy Vocab with custom vectors
vocab = nlp.vocab

# Copy word vectors from Gensim model to spaCy Vocab. Sliced to 1/30th size to prevent colab from crashing
for word, vector in zip(pretrained_w2v.index_to_key[:100000], pretrained_w2v.vectors[:100000]):
    vocab.set_vector(word, vector)

### Grade function

In [ ]:
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')  # Download the stopwords dataset if not already downloaded
stop_words = set(stopwords.words('english'))  # Get the list of English stop words

def preprocess_text(text):
    # Tokenization, lowercasing, and punctuation removal
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenize by space
    tokens = text.split()
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]  # Define stop_words if needed
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def short_comings(ans):
  words_not_in_model= []
  doc= nlp(ans)
  for token in doc:
      word = token.text.lower()  # Convert the word to lowercase for consistency
      if word not in pretrained_w2v:
          words_not_in_model.append(word)
  print(words_not_in_model)

In [ ]:
threshold= 0.79
TP, TN, FP, FN = 0, 0, 0, 0
def score_it(student_ans, mark_scheme, actual):
  global TP, TN, FP, FN # to make confusion matrix
  predicted= False
  for index, ans in enumerate(student_ans):
    ans= preprocess_text(ans) # preprocess response
    ans= nlp(ans)
    max_score=-1
    for ms_ans in mark_scheme:
      ms_ans= preprocess_text(ms_ans) # preprocess ms
      ms_ans= nlp(ms_ans)
      this_similarity= ans.similarity(ms_ans)
      max_score= max(this_similarity, max_score) # max used
    if max_score>threshold:
      predicted= True
    if actual and predicted:
        TP += 1
    elif not actual and predicted:
        FP += 1
    elif actual and not predicted:
        FN += 1
    else:
        TN += 1
    if not actual:
      index+=2
    print('response'+ str(index+1) + ': ' + str(max_score) + '\n\t   actual:' + str(actual) + ' predicted:' + str(predicted))

### Testing


In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/FYP TestData/FYPTesting.csv')
test.head(10)

,Question,Marking Scheme 1,Marking Scheme 2,Correct Answer 1,Correct Answer 2,Wrong Answer 1,Wrong Answer 2
0,What is data mining?,Uncovering patterns and insights within extens...,The goal of data mining is to reveal patterns ...,Data mining involves uncovering patterns and i...,Data mining is the process of discovering patt...,Data mining is a method of data storage.,Data mining is a form of data encryption.
1,What is the main goal of data mining?,Data mining aims to unearth valuable insights ...,Extracting valuable insights from data is the ...,The primary objective of data mining is to une...,To extract valuable knowledge from data.,To create new data.,To improve internet speed.
2,What is outlier detection in data mining?,Identifying unusual or uncommon data points wi...,The task of outlier detection is to recognize ...,Outlier detection entails recognizing unusual ...,Outlier detection is the identification of unu...,Outlier detection is used for data duplication.,Outlier detection is used for data compression.
3,What is the difference between supervised and ...,Supervised learning involves training the mode...,Labeled data is used to train the model in sup...,"In supervised learning, the model is trained w...","In supervised learning, the model is trained w...","Supervised learning uses unsupervised data, an...",There is no difference between supervised and ...
4,What is the purpose of data visualization in d...,Using visual representations to enhance unders...,The goal of data visualization is to represent...,Data visualization is employed to represent da...,Data visualization is used to present data gra...,Data visualization is used to generate random ...,Data visualization is used for data destruction.
5,What is the importance of domain knowledge in ...,Selecting pertinent data and interpreting mini...,The utilization of domain knowledge helps in c...,Domain knowledge aids in the selection of pert...,Domain knowledge helps in selecting relevant d...,Domain knowledge is only useful for playing vi...,Domain knowledge is used to confuse data miners.
6,What is the purpose of clustering in data mining?,Grouping similar data points based on their at...,Clustering is utilized to categorize akin data...,Clustering is utilized to group akin data poin...,Clustering is used to group similar data point...,Clustering is a way to sort data alphabetically.,Clustering is used for data multiplication.
7,What is the term for finding hidden patterns o...,Discovering hidden data patterns without prior...,Unsupervised learning focuses on uncovering co...,Unsupervised learning involves uncovering hidd...,Unsupervised learning is the term for finding ...,Randomization technique,Data duplication
8,What is cross-validation in data mining?,Evaluating the performance of predictive model...,Cross-validation is a method used to assess th...,Cross-validation is a method for evaluating th...,Cross-validation is a technique to assess the ...,Cross-validation is used to combine unrelated ...,Cross-validation is a data compression method.
9,What is the goal of text mining?,Text mining endeavors to unearth valuable insi...,The goal of text mining is to uncover valuable...,Text mining strives to extract valuable insigh...,Text mining aims to extract valuable informati...,Text mining is a way to store text data effici...,Text mining is used to create random text.


In [ ]:
test["Correct Answer 1"][i]

'Online data mining provides immediate insights from real-time data, whereas offline data mining analyzes historical data for deeper insights.'

In [ ]:
short_comings(test["Correct Answer 1"][i])

['-', ',', '.']


In [ ]:
preprocess_text(test["Correct Answer 1"][i])

'online data mining provides immediate insights realtime data whereas offline data mining analyzes historical data deeper insights'

In [ ]:
short_comings(preprocess_text(test["Correct Answer 1"][i]))

[]


In [ ]:
for i in range(test.shape[0]):
  print(f'Question{i+1}: {test["Question"][i]}')
  score_it([test["Correct Answer 1"][i], test["Correct Answer 2"][i]], [test["Marking Scheme 1"][i], test["Marking Scheme 2"][i]], True)
  score_it([test["Wrong Answer 1"][i], test["Wrong Answer 2"][i]], [test["Marking Scheme 1"][i], test["Marking Scheme 2"][i]], False)
  print('\n')

Question1: What is data mining?
response1: 0.96966946404275
	   actual:True predicted:True
response2: 0.8756954264501281
	   actual:True predicted:True
response3: 0.705812281163868
	   actual:False predicted:False
response4: 0.6745869140451819
	   actual:False predicted:False


Question2: What is the main goal of data mining?
response1: 0.9484433267584529
	   actual:True predicted:True
response2: 0.8123370415051399
	   actual:True predicted:True
response3: 0.6424386775316588
	   actual:False predicted:False
response4: 0.3148758277478591
	   actual:False predicted:False


Question3: What is outlier detection in data mining?
response1: 0.9559150988587714
	   actual:True predicted:True
response2: 0.9206041307725612
	   actual:True predicted:True
response3: 0.7519522391504855
	   actual:False predicted:False
response4: 0.7320161792882742
	   actual:False predicted:False


Question4: What is the difference between supervised and unsupervised learning in data mining?
response1: 0.94086380052

In [ ]:
confusion_matrix = [[TN, FP], [FN, TP]]

print("\nConfusion Matrix with threshold 0.79:")
for row in confusion_matrix:
    print(row)


Confusion Matrix with threshold 0.79:
[112, 12]
[10, 114]


In [ ]:
confusion_matrix = [[TN, FP], [FN, TP]]

print("\nConfusion Matrix with threshold 0.8:")
for row in confusion_matrix:
    print(row)


Confusion Matrix with threshold 0.8:
[114, 10]
[12, 112]


In [ ]:
confusion_matrix = [[TN, FP], [FN, TP]]

print("\nConfusion Matrix with threshold 0.75:")
for row in confusion_matrix:
    print(row)


Confusion Matrix with threshold 0.75:
[101, 23]
[4, 120]


In [ ]:
confusion_matrix = [[TN, FP], [FN, TP]]

print("\nConfusion Matrix with threshold 0.7:")
for row in confusion_matrix:
    print(row)


Confusion Matrix with threshold 0.7:
[85, 39]
[3, 121]
